In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data =pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv', parse_dates=['Date'])
data.head()

In [ ]:
# data 형태 확인
data.shape

* 각 column을 확인하여 dtype과 null값 확인

In [ ]:
data.info()

In [ ]:
data.isna().sum() 

In [ ]:
data.describe()

# **1. 결측률이 15%이상인 data filtering**

*   missing_data를 통해 결측치를 확인

*   통상적으로 결측률이 15%이상이면 특성을 제외하고 분석하는 것이 일반적

In [ ]:
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum() / data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.T

In [ ]:
dropList = list(missing_data[missing_data['Percent'] > 0.15].index)
data.drop(dropList, axis=1, inplace=True)

# **2. Location One-Hot Encoding**

**dtype이 object인 컬럽에 대해 먼저 조사한 이후**
*   categorical columns에 대해서는 LabelEncoding
*   Binary columns에 대해서는 LabelBinarizing

In [ ]:
# data columns마다 조사하여 dtype이 object인 컬럼명을 추출
objectColumns = [name for name in data.columns if data[name].dtype == object]

# 'RainToday', 'RainTomorrow'는 0 or 1로 구성되므로 BinaryCloumns으로 따로 분류
BinaryCloumns = objectColumns[-2:]
objectColumns = objectColumns[:-2]
print(objectColumns, BinaryCloumns)

In [ ]:
# Nan을 제외한 LabelEncoding을 진행하고 dtype 변환, 각 columns의 Label 저장
for col in objectColumns:
    Label = LabelEncoder()
    notnull = data[data[col].notnull()]
    isna = data[data[col].isna()]
    notnull[col] = Label.fit_transform(notnull[col])
    globals()['{}_col'.format(col)] = Label.classes_
    data = pd.concat([isna, notnull])
    data[col] = data[col].astype(float)

# Nan을 제외한 Binary Encoding을 진행하고 dtype 변환, 각 columns의 Label 저장
for col in BinaryCloumns:
    Binary = LabelBinarizer()
    notnull = data[data[col].notnull()]
    isna = data[data[col].isna()]
    notnull[col] = Binary.fit_transform(notnull[col])
    globals()['{}_col'.format(col)] = Label.classes_
    data = pd.concat([isna, notnull])
    data[col] = data[col].astype(float)


# 잘 되었는지 확인

data.info()

In [ ]:
# datetime순으로 DataFrame 정렬
data.sort_values(by=['Date'], inplace=True)

In [ ]:
# heatmap을 통해 각 columns 간 corr을 확인해봅니다.
# 뭐라도 건질 줄 알았는데 건진게 없습니다.
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(data.corr(), annot=True, fmt='.2f')

# Rainfall과 RainToday의 상관관계가 높아 확인을 해봅니다.


*   비오는 날과 오지 않는 날의 강수량이 1.0을 기준으로 나뉘는 것을 알 수 있습니다.

In [ ]:
print('비오는 날 최소 강수량 : {}mm \n화창한 날 최대 강수량 : {}mm'.format(data[data.RainToday.values == 1.0].Rainfall.min(), data[data.RainToday.values == 0.0].Rainfall.max()))

*   위 결과를 바탕으로 Rainfall의 값에 따라 RainToday을 집어 넣어줍니다.
*   또한, Rainfall과 RainToday의 Nan의 값이 동일 한 것으로 보아 RainToday의 값이 Nan인 row를 drop해줍니다.

In [ ]:
data.loc[(data.Rainfall.values > 1.0) & (data.Rainfall.isna()), 'RainToday'] = 1.0
data.loc[(data.Rainfall.values <= 1.0) & (data.Rainfall.isna()), 'RainToday'] = 0.0
data.RainToday.dropna(inplace=True)

* RainToday를 target으로 하기 위해 DataFrame을 분리하는 작업을 수행합니다.
* data_x의 shape을 확인하여 input columns를 확인합니다.

In [ ]:
data.fillna(0, inplace=True)

In [ ]:
data_x = data.drop('RainToday', axis=1)
data_y = data[['RainToday']]
data_x.shape

imputer = SimpleImputer(strategy='median')
imputeCol = data_x.iloc[:, 1:]
imputeCol = pd.DataFrame(imputer.fit_transform(imputeCol), columns=imputeCol.columns, index=list(imputeCol.index.values))
data_x = pd.concat([data_x[['Date']], imputeCol], axis=1)

In [ ]:
minmax = MinMaxScaler()
minmaxCol = data_x.iloc[:, 1:]
minmaxCol = pd.DataFrame(minmax.fit_transform(minmaxCol), columns=minmaxCol.columns, index=list(minmaxCol.index.values))
data_x = pd.concat([data_x[['Date']], minmaxCol], axis=1)

In [ ]:
data_x['Year'] = data_x['Date'].dt.year
data_x['Month'] = data_x['Date'].dt.month
data_x['Day'] = data_x['Date'].dt.day
data_x.drop(['Date'], axis=1, inplace=True)

* 데이터 분석을 시작하기 전에 train_set과 test_set을 나누어 줍니다.
* BATCH_SIZE와 EPOCHS, wieght를 저장할 filename을 먼저 선업해줍니다!

In [ ]:
data_x.reset_index(drop=True, inplace=True)

In [ ]:
data_x

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


regression = LogisticRegression(solver='liblinear', n_jobs=-1, random_state=0)

regression.fit(x_train, y_train)

In [ ]:
y_pred = regression.predict(x_test)
y_pred

In [ ]:
print('accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))